In [ ]:
import json
import os
from datasets import load_dataset


ds = load_dataset("PrimeIntellect/verifiable-coding-problems", split="train", trust_remote_code=True)
print(ds[0].keys())

In [ ]:
import ast
from pprint import pprint
dataset = []
for entry in ds:
    gold_standard_solution = entry["gold_standard_solution"]
    if gold_standard_solution is None:
        continue
    if gold_standard_solution.startswith("```python") and gold_standard_solution.endswith("```"):
        tests = entry["verification_info"]        
        if isinstance(tests, str):
            try:
                tests = ast.literal_eval(tests)
            except (ValueError, SyntaxError) as e:
                #Try Json loads instread
                try: 
                    tests = json.loads(entry["verification_info"])
                except (json.JSONDecodeError, SyntaxError, ValueError) as e:
                    print(repr(entry["verification_info"]))
                    print(f"Error in json.loads: {e}")
                    continue
        assert isinstance(tests, dict), "Tests should be a dictionary"
        assert tests['language'] == 'python'
        tests = tests["test_cases"]
        if len(tests) <= 4:
            continue 
        new_entry = {
            "problem": entry["prompt"],
            "solutions": [gold_standard_solution],
            "tests":tests,
        }

        # Assert tests is a dictionary 
        assert isinstance(tests, list), "Tests should be a dictionary"
        assert "input" in tests[0], "Tests should have an input key"
        assert "output" in tests[0], "Tests should have an output key"

        dataset.append(new_entry)

print(len(dataset))
print(dataset[0])
dataset = dataset
output_dir = os.path.abspath("../../train/code")
output_file = os.path.join(output_dir, "primeintellect.json")
with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)